# Extraction of Data From Zipped Files
**Overview:** This File will have the function Zip_Extractor(place) which will extract the data from the zipped files. 
    

In [8]:
import pandas as pd
import janitor
from zipfile import ZipFile 
import os

In [9]:
#Global Variables

#keys: State_typeOfFile        Item: DataFrame
place_dic = {}

In [10]:
states_dict = {
    "Alaska" : "VM2--AK--2020-03-18",
    "Alabama" : "VM2--AL--2020-02-24,",
    "Arkansas" : "VM2--AR--2020-02-24",
    "Arizona": "VM2--AZ--2020-02-19", 
    "California": "VM2--CA--2020-03-25", 
    "Colorado": "VM2--CO--2020-02-26",
    "Connecticut": "VM2--CT--2020-03-26",
    "DC": "VM2--DC--2020-03-02",
    "Delaware": "VM2--DE--2020-03-30",
    "Florida": "VM2--FL--2020-07-30",
    "Georgia": "VM2--GA--2020-03-02",
    "Hawaii": "VM2--HI--2020-03-02",
    "Iowa": "VM2--IA--2020-03-03",
    "Idaho": "VM2--ID--2020-03-27",
    "Illinois": "VM2--IL--2020-03-03",
    "Indiana": "VM2--IN--2020-02-27",
    "Kansas": "VM2--KS--2020-03-18",
    "Kentucky": "VM2--KY--2020-02-26",
    "Louisiana": "VM2--LA--2020-02-27",
    "Massachusetts": "VM2--MA--2020-02-19",
    "Maryland": "VM2--MD--2020-02-28",
    "Maine": "VM2--ME--2020-02-24",
    "Michigan": "VM2--MI--2020-03-02",
    "Minnesota": "VM2--MN--2020-02-25",
    "Missouri": "VM2--MO--2020-03-05",
    "Mississippi": "VM2--MS--2020-03-20",
    "Montana": "VM2--MT--2020-03-14",
    "North Carolina": "VM2--NC--2020-02-29",
    "North Dakota": "VM2--ND--2020-02-28",
    "Nebraska": "VM2--NE--2020-03-18",
    "North Hampshire": "VM2--NH--2020-03-03",
    "New Jersey": "VM2--NJ--2020-02-26",
    "New Mexico": "VM2--NM--2020-02-24",
    "Nevada": "VM2--NV--2020-02-22",
    "New York": "VM2--NY--2020-03-05",
    "Ohio": "VM2--OH--2020-02-28",
    "Oklahoma": "VM2--OK--2020-02-25",
    "Oregon": "VM2--OR--2020-02-25",
    "Philadelphia": "VM2--PA--2020-03-20",
    "Rhode Island": "VM2--RI--2020-02-28",
    "South Carolina": "VM2--SC--2020-02-21",
    "South Dakota": "VM2--SD--2020-02-25",
    "Tennessee": "VM2--TN--2020-03-31",
    "Texas": "VM2--TX--2020-03-02",
    "Utah": "VM2--UT--2020-04-07",
    "Virginia": "VM2--VA--2020-03-01",
    "Vermont": "VM2--VT--2020-02-27",
    "Washington": "VM2--WA--2020-04-20",
    "Wisconsin": "VM2--WI--2020-03-21",
    "West Virginia": "VM2--WV--2020-03-29",
    "Wyoming": "VM2--WY--2020-03-02"
}
len(states_dict)

51

In [11]:
def zip_extractor(place):
    # if there's an error, try \ instead of /
    file_name = "//storage.rcs.nyu.edu/L2_Political/03-01-2020-delivery/files-by-state/"+ place + ".zip"
    # opening the zip file in READ mode 
    zip = ZipFile(file_name,'r') 

    demographics_file = zip.open(place + "-DEMOGRAPHIC.tab")
    vote_history_file = zip.open(place + "-VOTEHISTORY.tab")
    zip.close()
    demographics = pd.read_csv(demographics_file, 
                                sep='\t', dtype=str, encoding='unicode_escape',
                               nrows=100)
    vote_history = pd.read_csv(vote_history_file, 
                                sep='\t', dtype=str, encoding='unicode_escape',
                                nrows=100)
    
    return demographics, vote_history

In [12]:
#Iterates through all of the files by state and unzips 
def create_df(): 
    for file in os.listdir('//storage.rcs.nyu.edu/L2_Political/03-01-2020-delivery/files-by-state/'):
        rename = file[file.find("--")+2:file.find("-2")-1]

        place = file[:file.find(".zip")]
        demographics, vote_history = zip_extractor(place)

        place_dic["%s_demographic" %rename] = demographics
        place_dic["%s_voting_history" %rename] = vote_history


In [13]:
def get_state_keys(): 
    return place_dic.keys()
get_state_keys()

dict_keys([])

# Removing Null

In [ ]:
#Remove null with threshold of 95% 
def remove_null(): 
    for key in place_dic.keys(): 
                        #finds 95% of columns 
        threshold = int(place_dic[key].shape[1] * 0.75)
        place_dic[key].dropna(thresh=threshold)
        
remove_null()


# Sampling Methods

**Simple Random Sampling** everymemeber of the population has an equal chance of being selected 

**Stratified Sampling** Dividing the population into subpopulations that may differ in important ways. It allows you to draw more precise conclussions by measuring that each subgroup is represented.

**Clustering** Dividing the population into subgroups, but each subgroup has similar characteristics. Randomly select the subgroups instead of sampling individuals from subgroup.

In [11]:
def merge(state_demographic, state_vote_history):
    merged_file = pd.merge(state_vote_history, state_demographic,
                       how='left', left_on='LALVOTERID', right_on='LALVOTERID')
    
    return merged_file

In [52]:
#TODO: should have more types of sampling; with/without replacement etc
def get_sample(file, sample_size):
    return file.sample(frac=sample_size)

#stratified sampling
#choose equal sample sizes from each 
def build_stratified_national_sample(sample_size):
    national_sample = get_sample(AK, sample_size)
    for df in place_dic.items():
        national_sample = pd.concat(get_sample(df, sample_size), axis=0)
    print(res)
    
#simple random sampling
#choose randomly from national dataset
def build_simple_random_national_sample(sample_size):
    total_rows = 0
    for df in place_dic.items():
        total_rows.append(len(df))
    national_sample = get_sample(AK, len(AK)/total_rows)
    for file in files:
        national_sample = pd.concat([national_sample, get_sample(file, len(file)/total_rows)], axis = 0)
        

In [ ]:
#Clustering based on ____input____
def cluster_on_input(column): 
    pass
    
#TODO K-means clustering 
    

In [54]:
#calling the Sampling Functions
merged_CT = merge(place_dic["CT_demographic"], place_dic["CT_voting_history"])

ex_sample = get_sample(place_dic["CT_demographic"],0.1)

build_stratified_national_sample(10)


NameError: name 'AK' is not defined

# What is a Better Predictors


In [ ]:
#Simple Correlations

#Pearson

#Expects two columns to compare
def correlation(column1, column2):
    return place_dic[column1].corr(place_dic[column2])
    
#Spearman
def spearman_corr(column1, column2): 
    return place_dic[column1].corr(place_dic[column2], method='spearman')


# Ideas  for What can tell from data 

 - Is there any evidence that _______ impacts ______ or ______ ?
 - balance checking in areas
 - linear regression
 - some tests from Causal Inference